In [1]:
# !pip install cvxopt
import cvxpy as cp

In [ ]:
import numpy as np

# Example 1

In [2]:
# Reproduce the example from `https://www.cvxpy.org/index.html`.

# Problem data.
m = 30
n = 20
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# Construct the problem.
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A @ x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print(constraints[0].dual_value)

[-1.79109255e-19  2.85112420e-02  2.79973443e-19  3.37658729e-20
 -2.72802663e-19  1.49285011e-01 -9.94082533e-20  8.35373900e-20
  2.46718649e-01  5.78224144e-01 -4.03739463e-19  1.01242860e-03
 -9.28486180e-20  2.26767464e-01 -1.58813678e-19 -8.97232272e-20
 -1.22145729e-19 -1.51509428e-19  1.12060672e-19 -3.48318635e-19]
[ 2.50938945  0.          2.78354615  1.79425782 13.08579183  0.
  0.73716363  3.35344995  0.          0.          8.93825054  0.
  7.02955161  0.          4.71068649  3.18873635  2.06090107 10.08166738
  3.0481157   8.53268239]


In [8]:
# https://www.cvxpy.org/tutorial/advanced/index.html#getting-the-standard-form
pd1, pd2, pd3 = prob.get_problem_data(cp.CVXOPT)

In [5]:
pd1

{'param_prob': <cvxpy.reductions.dcp2cone.cone_matrix_stuffing.ParamConeProg at 0x7f97b06ad850>,
 'dims': (zero: 0, nonneg: 40, exp: 0, soc: [32], psd: [], p3d: []),
 'c': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]),
 'A': None,
 'b': None,
 'G': <72x21 sparse matrix of type '<class 'numpy.float64'>'
 	with 642 stored elements in Compressed Sparse Column format>,
 'h': array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1

In [6]:
pd2

Chain(reductions=[<cvxpy.reductions.dcp2cone.dcp2cone.Dcp2Cone object at 0x7f97b06bb090>, <cvxpy.reductions.cvx_attr2constr.CvxAttr2Constr object at 0x7f97b06bbf10>, <cvxpy.reductions.dcp2cone.cone_matrix_stuffing.ConeMatrixStuffing object at 0x7f97b06bb190>, <cvxpy.reductions.solvers.conic_solvers.cvxopt_conif.CVXOPT object at 0x7f97b0388b50>])

In [7]:
pd3

 (),
 {'var_id': 66,
  'dims': (zero: 0, nonneg: 40, exp: 0, soc: [32], psd: [], p3d: []),
  'eq_constr': [],
  'other_constr': [NonNeg(Expression(AFFINE, UNKNOWN, (20,))),
   NonNeg(Expression(AFFINE, UNKNOWN, (20,))),
   SOC(Expression(AFFINE, UNKNOWN, (1,)))],
  'offset': array(0.)}]

# Example 2

In [34]:
# https://druce.ai/2020/12/portfolio-opimization

In [35]:
# Problem data.
m = 100
n = 20
np.random.seed(1)
historical_mu = np.random.randn(m, n)

In [18]:
mu = np.random.randn(n)

In [27]:
# The `transpose()` is necessary.
covariance = np.cov(historical_mu.transpose())

In [24]:
weights = cp.Variable(n)

In [25]:
rets = mu.T @ weights

In [50]:
portfolio_variance = cp.quad_form(weights, covariance)

In [51]:
minimize_variance = cp.Problem(
    cp.Minimize(portfolio_variance), [cp.sum(weights) == 1]
)

In [52]:
minimize_variance.solve()

0.03149854482732454

In [53]:
rets.value

-0.061490906784330925

In [42]:
weights.value

array([0.04577215, 0.04920539, 0.0221771 , 0.06538957, 0.0103917 ,
       0.01044984, 0.06562787, 0.07408221, 0.05158417, 0.08106313,
       0.03762758, 0.10165571, 0.03088424, 0.08039891, 0.05895037,
       0.076406  , 0.0082815 , 0.02650216, 0.05957543, 0.04397498])

In [54]:
maximize_returns = cp.Problem(
    cp.Maximize(rets),
    [
        cp.sum(weights) == 1,
        # Long-short will try to increase leverage to infinity (and fail to converge)
        weights >= 0,
    ],
)

In [55]:
maximize_returns.solve()

1.4969952468999923

In [56]:
portfolio_variance.value

1.2686101771457268

In [57]:
maximize_returns_2 = cp.Problem(
    cp.Maximize(rets),
    [
        cp.norm(weights) <= 1.5,
        cp.sum(weights) == 1,
        portfolio_variance <= 0.05,
    ],
)

In [58]:
maximize_returns_2.solve()

0.5737763507055227

In [63]:
portfolio_variance.value

0.049999999968532514

# More examples

In [33]:
# https://nbviewer.org/github/cvxgrp/cvx_short_course/blob/master/applications/portfolio_optimization.ipynb

In [ ]:
# https://github.com/druce/portfolio_optimization/blob/master/Portfolio%20optimization.ipynb